In [ ]:
# https://t.me/russica2

import pandas as pd
import json
from pprint import pprint

# day today
from datetime import date
today = date.today()



In [ ]:
file_path = "/Users/Oksana/Downloads/Telegram Desktop/ChatExport_2024-06-01 (2)/result.json"
chat_name = "НЕЗЫГАРЬ"
# Open the file and load the data
with open(file_path, "r") as file:
    data = json.load(file)

#pprint(data)

In [ ]:
# build a dataframe and unnest the json 'messages' column
df = pd.json_normalize(data, record_path =['messages'])

# ignore the first row
df = df.iloc[1:]
display(df.head(3))

pprint(df.columns)

In [ ]:
# split time_tt to date and time
df['date'] = df['date'].str.split(' ').str[0]
df['time'] = df['date'].str.split(' ').str[1]

# # Extract both month and year into one column
df['month_year'] = pd.to_datetime(df['date']).dt.to_period('M')
df['month_year'] = df['month_year'].astype(str)
display(df.head(3))

In [ ]:
# Function to extract plain text
def extract_plain_text(row):
    return ''.join([item if isinstance(item, str) else item['text'] for item in row])

# Apply function to DataFrame
df['plain_text'] = df['text'].apply(extract_plain_text)

In [ ]:
### TEXT + MEDIA in Messages ###

# group by month and year and count number of posts
df_count_text = df.groupby('month_year')['text'].count()
df_count_text = df_count_text.reset_index()

# create a line plot, where x-axis is the month and year and y-axis is the number of messages
df_count_text.plot.line(x='month_year', y='text', figsize=(15, 6), title='Number of text and media messages per month')

# plot with dots
df_count_text.plot(x='month_year', y='text', kind='scatter', figsize=(15, 6), title='Number of text and media messages per month')

# bar chart
df_count_text.plot.bar(x='month_year', y='text', figsize=(15, 6), title='Number of text and media messages per month')


In [ ]:

import re
## Extract names from text 
df_ru_names = pd.read_csv('/Users/Oksana/Documents/ALEPH/RU/ru_tg_channels/telegram-api/ru_names.csv')       
names_list_male = df_ru_names['ru_male_name'].tolist()
names_list_female = df_ru_names['ru_female_name'].tolist()
names_list = names_list_male + names_list_female
print(names_list)
type(names_list)
# remove None values
names_list = [x for x in names_list if str(x) != 'nan']
#names_list = [x for x in names_list if str(x) != 'None']

In [ ]:
### FIND MALE AND FEMALE NAMES IN TEXT ###

import re
## Extract names from text 
df_ru_names = pd.read_csv('/Users/Oksana/Documents/ALEPH/RU/ru_tg_channels/telegram-api/ru_names.csv')       
names_list_male = df_ru_names['ru_male_name'].tolist()
names_list_female = df_ru_names['ru_female_name'].tolist()
# remove None values
names_list_female = [x for x in names_list_female if str(x) != 'nan']
names_list = names_list_male + names_list_female

# Function to find names and surnames in text

def find_names(text, names_list):
    found_names = []
    for name in names_list:
        # Regex to find name possibly followed by a surname
        """
        not extracting a surname before a name, improve in the future
        """
        #pattern = r'\b' + re.escape(name) + r'\s+[А-Я][а-я]+' 
        #pattern = rf'\b{name}\b(?:\s+[А-Я][а-я]+)?'
        #pattern = rf'\b{name}\s+[А-Я][а-я]+'
        #pattern = rf'\b{name[:-1]}[а-яё]*\s+[А-ЯЁ][а-яё]+\b' # name + surname, helps with names with different endings
        #pattern = rf'\b{name[:-1]}[а-яё]*\s+[А-ЯЁ][а-яё]+\b'
        pattern = rf'\b{name[:-1]}[а-яё]*\s+[А-ЯЁ][а-яё]+\b'
        matches = re.findall(pattern, text)

        found_names.extend(matches)
    return found_names

# Apply the function to each row in df['text']
df['plain_text'] = df['plain_text'].fillna('')

df['extracted_names'] = df['plain_text'].apply(lambda x: find_names(x, names_list))

# Display the result
#display(df.head(2))


#df.to_csv(f'extracted_names_{chat_name}_{today}.csv', index=False)


In [ ]:
# In df select id, date, time, month_year, plain_text, extracted_names
df_selected_names = df[['id', 'date', 'time', 'month_year', 'plain_text', 'extracted_names']]
df_selected_names.to_csv(f'extracted_names_{chat_name}_{today}_v2.csv', index=False)

In [ ]:
# count empty rows without text and names in extracted_names column
empty_rows = df_selected_names[df_selected_names['extracted_names'].apply(lambda x: len(x) == 0)]
print(len(empty_rows))

# count total number of names extracted
total_names = df_selected_names['extracted_names'].apply(lambda x: len(x)).sum()
print(total_names)

# count number of rows in the column extracted_names
total_rows = df_selected_names['extracted_names'].apply(lambda x: len(x)).count()
print("Total number of rows in the column extracted_names:")
print(total_rows)

# count not empty rows in the column extracted_names
not_empty_rows = df_selected_names[df_selected_names['extracted_names'].apply(lambda x: len(x) > 0)]
print("Total number of not empty rows in the column extracted_names:")
print(len(not_empty_rows))


In [ ]:
len(df)

In [ ]:
df_short = df.head(3000)

In [ ]:
print(df_short.columns)

In [ ]:
# select columns id, date, time, month_year, plain_text, extracted_names
df_short_col = df[['id', 'date', 'month_year', 'plain_text', 

                     'from', 'from_id',  # same, but names and ID
                     'forwarded_from']]

display(df_short_col.head(2))

In [ ]:
### FORWARDERED MESSAGES ###

# filter  out rows with nan in 'forwarded_from' column
df_short_col_forwarded = df_short_col[df_short_col['forwarded_from'].notna()]

In [ ]:
# count the number of unique values in the column 'Forwarder'
print("Number of unique forwarders:", df_short_col_forwarded['forwarded_from'].nunique())

# Count how many times each unique value occurs in the column 'Forwarder'
print("Number of times each forwarder occurs:")
display(df_short_col_forwarded['forwarded_from'].value_counts())

# create a new temp dataframe where value_counts() is stored
temp = df_short_col_forwarded['forwarded_from'].value_counts().reset_index()
temp.columns = ['forwarded_from', 'Count']

In [ ]:
xy_gr = df_short_col_forwarded.groupby(['from', 'forwarded_from'])['forwarded_from'].count()
xy_gr = xy_gr.to_frame(name="count").reset_index()


### Who communicates the most between each others 
display(xy_gr.sort_values(by='count', ascending=False).head(5))



In [ ]:
# Group the df by 'Sender' and 'Frowarder' and count the number of times each unique value occurs in the column 'Forwarder'
print("Number of times each sender forwards a message to a forwarder:")
display(df_short_col_forwarded.groupby(['from', 'forwarded_from']).size().reset_index(name='count'))

# create a new temp dataframe where this is stored
temp2 = df.groupby(['from', 'forwarded_from']).size().reset_index(name='Count')

# Build a network graph using the 'networkx' library
import networkx as nx
import matplotlib.pyplot as plt

# create a directed graph with the 'networkx' library from the 'temp2' dataframe
G = nx.from_pandas_edgelist(temp2, 'from', 'forwarded_from', ['Count'], create_using=nx.DiGraph())

# draw the graph
#plt.figure(figsize=(10, 10))

# nx.draw(G, with_labels=True, node_size=100, node_color='red', 
#            font_size=5, font_weight='bold', edge_color='gray', 
#            pos=nx.spring_layout(G))

# a different way to draw the graph
# nx.draw_networkx(G, with_labels=True, node_size=100, node_color='red',
#                  font_size=5, font_weight='bold', edge_color='gray',
#                  pos=nx.spring_layout(G))



In [ ]:
# Count numbver of nodes and edges in the graph
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

In [ ]:
# Create an edge dataframe from the graph
edge_df = nx.to_pandas_edgelist(G)
print(edge_df)

# Save the edge dataframe to a csv file
edge_df.to_csv(f'edge_list_{chat_name}_{today}.csv', index=False)

In [ ]:
### TEXT ANALYSIS ###

# filter out empty rows in the column 'plain_text'
df_forwarded = df_short_col_forwarded[df_short_col_forwarded['plain_text'].notna()]
df_forwarded

In [ ]:
### Text Analysis ###

### TEXT ANALYSIS ###

### NLTK libraries
import nltk
import ssl

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# https://www.youtube.com/watch?v=IBmZAYR0pns

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download('all')

In [ ]:
# lower text
df_text_ta = df_forwarded.copy()
df_text_ta['plain_text'] = df_text_ta['plain_text'].str.lower()

# Tokenize text
df_text_ta['text_tokens'] = df_text_ta['plain_text'].apply(word_tokenize)
display(df_text_ta.head(4))

In [ ]:
### Stopwords
from stopwords_emails_ru import stopwords_merged
print(len(stopwords_merged), type(stopwords_merged), stopwords_merged)
stopwords_merged = pd.DataFrame(stopwords_merged)
stopwords_merged

In [ ]:
stopwords = pd.read_csv('/Users/Oksana/Documents/ALEPH/RU/ru_tg_channels/telegram-api/stopwords_merged.csv')
display(stopwords.head(3))

In [ ]:
df_text_ta['text_tokens_WO_stopwords'] = df_text_ta['text_tokens'].apply(lambda x: [item for item in x if item not in stopwords.values])
display(df_text_ta.head(2))

# remove ' from text
df_text_ta['text_tokens_WO_stopwords'] = df_text_ta['text_tokens_WO_stopwords'].apply(lambda x: [item for item in x if item != "'"])
df_text_ta

In [ ]:
# Count the number of words in emails_grouped['bodyText_tokenized_filtered'] column
print("The dataframe with a new column where the number of words in a text is counted")

df_text_ta['count_words'] = df_text_ta['text_tokens_WO_stopwords'].apply(lambda x: len(x))
display(df_text_ta.head(1))

In [54]:
### Lemmatize words 
from collections import Counter

lemmatizer = WordNetLemmatizer()
df_text_ta['words_lemmatized'] = df_text_ta['text_tokens_WO_stopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

print("\n" ,"The dataframe with a new column 'words_lemmatized_count' where the words are lemmatized and count")
df_text_ta['words_lemmatized_count'] = df_text_ta['words_lemmatized'].apply(lambda x: Counter(x))
display(df_text_ta.head(1))


 The dataframe with a new column 'words_lemmatized_count' where the words are lemmatized and count


,id,date,month_year,plain_text,from,from_id,forwarded_from,text_tokens,text_tokens_WO_stopwords,count_words,words_lemmatized,words_lemmatized_count
6,4160,2017-12-31T00:44:02,2017-12,блогер года — незыгарь (@russica2),НЕЗЫГАРЬ,channel1096463945,Russia calls,"[блогер, года, —, незыгарь, (, @, russica2, )]","[блогер, незыгарь, russica2]",3,"[блогер, незыгарь, russica2]","{'блогер': 1, 'незыгарь': 1, 'russica2': 1}"


In [55]:
display(df_text_ta.head(2))

df_text_ta.to_csv(f'text_analysis_{chat_name}_{today}.csv', index=False)

# select columns id, date, month_year, words_lemmatized_count
df_text_ta_selected = df_text_ta[['id', 'date', 'month_year', 'count_words', 'words_lemmatized', 'words_lemmatized_count']]
df_text_ta_selected.to_csv(f'text_analysis_{chat_name}_{today}_v2.csv', index=False)
display(df_text_ta_selected.head(2))

,id,date,month_year,plain_text,from,from_id,forwarded_from,text_tokens,text_tokens_WO_stopwords,count_words,words_lemmatized,words_lemmatized_count
6,4160,2017-12-31T00:44:02,2017-12,блогер года — незыгарь (@russica2),НЕЗЫГАРЬ,channel1096463945,Russia calls,"[блогер, года, —, незыгарь, (, @, russica2, )]","[блогер, незыгарь, russica2]",3,"[блогер, незыгарь, russica2]","{'блогер': 1, 'незыгарь': 1, 'russica2': 1}"
35,4229,2018-01-01T22:39:37,2018-01,#моменты,НЕЗЫГАРЬ,channel1096463945,Пяр во время чумы,"[#, моменты]",[моменты],1,[моменты],{'моменты': 1}


,id,date,month_year,count_words,words_lemmatized,words_lemmatized_count
6,4160,2017-12-31T00:44:02,2017-12,3,"[блогер, незыгарь, russica2]","{'блогер': 1, 'незыгарь': 1, 'russica2': 1}"
35,4229,2018-01-01T22:39:37,2018-01,1,[моменты],{'моменты': 1}


In [42]:
import pandas as pd
df_text_ta_selected = pd.read_csv(f'text_analysis_НЕЗЫГАРЬ_2024-06-02_v2.csv')
display(df_text_ta_selected.head(2))

,id,date,month_year,count_words,words_lemmatized,words_lemmatized_count
0,4160,2017-12-31T00:44:02,2017-12,3,"['блогер', 'незыгарь', 'russica2']","Counter({'блогер': 1, 'незыгарь': 1, 'russica2..."
1,4229,2018-01-01T22:39:37,2018-01,1,['моменты'],Counter({'моменты': 1})


In [45]:
# DF with most common words
df_lemma_mostCommon = df_text_ta_selected.set_index('month_year').words_lemmatized_count.apply(lambda x: pd.Series(x, dtype='object')).stack().reset_index()

# an additional filter in level_1
#df_lemma_mostCommon = df_lemma_mostCommon[df_lemma_mostCommon['level_1'] != '\'']
#df_lemma_mostCommon = df_lemma_mostCommon[df_lemma_mostCommon['level_1'] != "''"]
#df_lemma_mostCommon.to_csv(f'df_lemma_mostCommon_{chat_name}.csv')

df_lemma_mostCommon.columns = ['month_year', 'word', 'count']
df_lemma_mostCommon.head(2)


,month_year,word,count
0,2017-12,0,"Counter({'блогер': 1, 'незыгарь': 1, 'russica2..."
1,2018-01,0,Counter({'моменты': 1})


In [53]:
type(df_lemma_mostCommon['count'])

pandas.core.series.Series

In [44]:
# Convert month_year to datetime and sort
df_lemma_mostCommon['month_year'] = pd.to_datetime(df_lemma_mostCommon['month_year'], format='%Y-%m')
df_lemma_mostCommon = df_lemma_mostCommon.sort_values(by='month_year')
df_lemma_mostCommon.head(2)




,month_year,word,count
0,2017-12-01,0,"Counter({'блогер': 1, 'незыгарь': 1, 'russica2..."
1,2018-01-01,0,Counter({'моменты': 1})


In [49]:
# Group by month_year and word and sum the count
most_popular_words = df_lemma_mostCommon.groupby(['month_year', 'word'])['count'].sum().reset_index()
most_popular_words.head(2)

,month_year,word,count
0,2017-12,0,"Counter({'блогер': 1, 'незыгарь': 1, 'russica2..."
1,2018-01,0,"Counter({'моменты': 1})Counter({'моменты': 1, ..."


In [48]:
# Find the most popular word for each month-year
most_popular_words = df_lemma_mostCommon.loc[df_lemma_mostCommon.groupby('month_year')['count'].idxmax()]
most_popular_words.head(2)

TypeError: reduction operation 'argmax' not allowed for this dtype

In [50]:
# Plot the most popular word for each month-year
import matplotlib.pyplot as plt

# Convert back to string for plotting purposes
#most_popular_words['month_year'] = most_popular_words['month_year'].dt.strftime('%Y-%m')
# Plot the most popular word for each month-year
plt.figure(figsize=(12, 8))
plt.plot(most_popular_words['month_year'], most_popular_words['count'], marker='o', linestyle='-', markersize=10)

# Annotate each point with the corresponding word
for i, row in most_popular_words.iterrows():
    plt.annotate(row['word'], (row['month_year'], row['count']),
                 textcoords="offset points", xytext=(0,10), ha='center', fontsize=12, bbox=dict(facecolor='yellow', alpha=0.5))

plt.xlabel('Month-Year')
plt.ylabel('Count of Most Popular Word')
plt.title('Most Popular Word by Month-Year')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


/var/folders/gx/61q8sz7d4r9g0nl993whgmhc0000gp/T/ipykernel_15700/1854162901.py:19: UserWarning: Glyph 129300 (\N{THINKING FACE}) missing from current font.
  plt.tight_layout()
/var/folders/gx/61q8sz7d4r9g0nl993whgmhc0000gp/T/ipykernel_15700/1854162901.py:19: UserWarning: Glyph 128165 (\N{COLLISION SYMBOL}) missing from current font.
  plt.tight_layout()
/var/folders/gx/61q8sz7d4r9g0nl993whgmhc0000gp/T/ipykernel_15700/1854162901.py:19: UserWarning: Glyph 129318 (\N{FACE PALM}) missing from current font.
  plt.tight_layout()
/var/folders/gx/61q8sz7d4r9g0nl993whgmhc0000gp/T/ipykernel_15700/1854162901.py:19: UserWarning: Glyph 127995 (\N{EMOJI MODIFIER FITZPATRICK TYPE-1-2}) missing from current font.
  plt.tight_layout()
/var/folders/gx/61q8sz7d4r9g0nl993whgmhc0000gp/T/ipykernel_15700/1854162901.py:19: UserWarning: Glyph 129335 (\N{SHRUG}) missing from current font.
  plt.tight_layout()
/var/folders/gx/61q8sz7d4r9g0nl993whgmhc0000gp/T/ipykernel_15700/1854162901.py:19: UserWarning: Glyph 

ValueError: 
': 3, 'здания': 3, 'универмаг': 2, 'вернуть': 2, 'долг': 2, 'взыскание': 2, 'здание': 2, 'которое': 2, 'залоге': 2, 'начались': 2, 'суд': 2, 'стало': 2, 'краснова': 2, 'представители': 2, 'реестра': 2, 'недвижимости': 2, 'хажаев': 2, 'слову': 2, 'вектор': 2, 'права': 2, '🤬😱🤦🏻\u200d♂️🙄🤐': 1, 'подельниками': 1, 'похитили': 1, 'помощи': 1, 'простой': 1, 'незамысловатой': 1, 'схемы': 1, '2013': 1, 'собственник': 1, 'известного': 1, 'ленинском': 1, 'проспекте': 1, 'москве': 1, 'взяло': 1, 'росинтербанке': 1, 'кредит': 1, 'срок': 1, 'вернул': 1, '2015': 1, 'потребовал': 1, 'суде': 1, 'обратить': 1, 'криминальные': 1, 'чудеса': 1, 'проблемы': 1, 'цб': 1, 'ввёл': 1, 'временную': 1, 'администрацию': 1, 'буквально': 1, 'недели': 1, 'прихода': 1, 'юрист': 1, 'крескиян': 1, 'е.е': 1, 'пришёл': 1, 'отказался': 1, 'иска': 1, 'фактически': 1, 'простил': 1, 'повторно': 1, 'просудить': 1, 'невозможно': 1, 'доверенность': 1, 'благотворителю': 1, 'выдала': 1, 'пред.предправления': 1, 'марина

/Users/Oksana/Library/Python/3.10/lib/python/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129300 (\N{THINKING FACE}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/Users/Oksana/Library/Python/3.10/lib/python/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 128165 (\N{COLLISION SYMBOL}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/Users/Oksana/Library/Python/3.10/lib/python/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129318 (\N{FACE PALM}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/Users/Oksana/Library/Python/3.10/lib/python/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 127995 (\N{EMOJI MODIFIER FITZPATRICK TYPE-1-2}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/Users/Oksana/Library/Python/3.10/lib/python/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 129335 (\N{SHRUG}) missing from current font.

ValueError: 
': 3, 'здания': 3, 'универмаг': 2, 'вернуть': 2, 'долг': 2, 'взыскание': 2, 'здание': 2, 'которое': 2, 'залоге': 2, 'начались': 2, 'суд': 2, 'стало': 2, 'краснова': 2, 'представители': 2, 'реестра': 2, 'недвижимости': 2, 'хажаев': 2, 'слову': 2, 'вектор': 2, 'права': 2, '🤬😱🤦🏻\u200d♂️🙄🤐': 1, 'подельниками': 1, 'похитили': 1, 'помощи': 1, 'простой': 1, 'незамысловатой': 1, 'схемы': 1, '2013': 1, 'собственник': 1, 'известного': 1, 'ленинском': 1, 'проспекте': 1, 'москве': 1, 'взяло': 1, 'росинтербанке': 1, 'кредит': 1, 'срок': 1, 'вернул': 1, '2015': 1, 'потребовал': 1, 'суде': 1, 'обратить': 1, 'криминальные': 1, 'чудеса': 1, 'проблемы': 1, 'цб': 1, 'ввёл': 1, 'временную': 1, 'администрацию': 1, 'буквально': 1, 'недели': 1, 'прихода': 1, 'юрист': 1, 'крескиян': 1, 'е.е': 1, 'пришёл': 1, 'отказался': 1, 'иска': 1, 'фактически': 1, 'простил': 1, 'повторно': 1, 'просудить': 1, 'невозможно': 1, 'доверенность': 1, 'благотворителю': 1, 'выдала': 1, 'пред.предправления': 1, 'марина

<Figure size 1200x800 with 1 Axes>